The exoGravity package
======================

Installation
------------

### Requirements

-   Tested on Linux/Fedora 33, 34

-   Python 2.7 or Python 3.x, ideally with x\>=5 (required for
    ruamel.yaml)

-   Astropy (\>=2.0.2)

-   ruamel.yaml \>=0.16.5 (recommended) or alternatively pyyaml \>=3.11

-   NumPy (\>=1.16.5), Scipy (\>=0.19.0)

-   cleanGravity (see installation procedure below)

### Installation procedure

Download or clone cleanGravity: https://gitlab.obspm.fr/mnowak/cleanGravity
Add the directory to your python path, so that you can then import the package
in any of your python code: "import cleanGravity as cg"

Download or clone exoGravity, https://gitlab.obspm.fr/mnowak/exoGravity and make
all the following scripts executable (`chmod +x`): create_config.py, create_phase_reference.py, astrometry_reduce.py, spectrum_reduce.py, swap_reduce.py, normalize_spectrum.py, plot_spectrum.py.
Optionally, you can the directory to your path, so that you can execute these scripts from anywhere.


Usage
-----

The exoGravity package provides a number of scripts, which should be used
sequentially to reduce the data. 

The scripts are the following:

-   `create_phase_ref.py`: is used to extract the phase reference from the data, and to write it
    directly in the fits files.
    
-   `astrometry_reduce.py`: is used to extract the astrometry from the
    observations. The script will output the best astrometry on the
    terminal, and will add the astrometric solution for each `planet_oi`
    in the YAML configuration file.

-   `spectrum_reduce.py`: is used to extract the spectrum from the
    observations. The script will create a “contrast.txt” and
    “covariance.txt”, to store the final spectrum and its error bars.
    Ideally, an astrometric solution should be provided for each planet
    observation in the YAML configuration file. If not, the script will
    assume the fiber to be perfectly centered on the planet, which may
    lead to bad results.

-   `swap_reduce.py` is only used for off-axis observations, for which
    the astrometry of the reference binary is not known with enough
    precision. This script will extract he astrometric solution for the
    binary from the observations themselves.

These scripts use a common YAML configuration file, which describes the
data files, the reduction parameters, etc. The exoGravity package comes
with a tool to help you create a properly formatted YAML file:
`create_config.py`. This script takes at least one argument, which is
the path to the directory in which the astrored files are stored. It
goes through the files, identifies on-planet, on-star, and swap
observations, and created an associated config file. lease see the
documentation for further options.

To use these scripts, start by putting all the “astrored” files
corresponding to your observation (i.e. on-planet, on-star, possibly
swap reference) in a common directory. To test the scripts, you can download
a dataset from the exoGravity Nextcloud, for example the dataset obtained on
beta Pic c, on Mar, 10, 2020:
 LINK

Instead of creating a configuration file from scratch yourself, it is
strongly advised to use the script provided, and to tweak the parameters
afterwards. Run:

``` bash
create_config.py datadir=./2020-03-08/ output=betaPictorisc2020.yml
```

This will create the YAML file. You can now open it with a text editor.
For this demo, we will change a few parameters to accelerate the
reduction. First, in the `general` section, look for the `gofast`
parameter, and set it to `true`. Then, search for the number of points
in the RA/DEC maps (`n_ra` and `n_dec`, and set them to 50 instead of
the default 100). Note that these changes could also have been requested
at the creation of the file, by calling:

``` bash
create_config.py datadir=./2020-03-08/ output=betaPictorisc2020.yml gofast=True nra=50 ndec=50
```

The last change we will make is to skip the reduction of some of the
files. The YAML format supports commenting (with the char `#`). In the
`general` section of the configuration file, look for the `reduce`
keyword. This gives a list (one element per line, preceded by a dash, as
per YAML specification) of keys to the planet files that will be
reduced. The keys refer to the elements of the `planet_ois` section of
this same configuration file. For this example, we will only keep the
first 5 files.

The `general` section of you config file should now look something like:

    general:
      contrast_file: null
      datadir: /data1/gravity/BetaPictoris/2018-09-22/reduced/
      declim:
      - 117.47058823529412
      - 127.47058823529412
      gofast: true
      n_dec: 50
      n_opd: 100
      n_ra:50
      noinv: false
      phaseref_mode: DF_STAR
      ralim:
      - 65.0
      - 75.0
      reduce:
      - p0
      - p1
      - p2
      - p3
      - p4
      - p5
    #  - p6
    #  - p7
    #  - p8
    #  - p9
    #  - p10
    #  - p11
      save_fig: true
      star_diameter: 0.0
      star_order: 3

Now we can create the phase reference from the on-star files, which will be used
to correct the on-planet observations.

``` bash
create_phase_reference.py config_file=betaPictorisc2020.yml
```

And then calculate the astrometry of the planet:

``` bash
astrometry_reduce.py config_file=betaPictorisc2020.yml
```

At the end, the script will output some information:

```
[INFO]: t=157.20s RA: -70.96+-2.059 mas
[INFO]: t=157.20s DEC: -120.44+-1.930 mas
[INFO]: t=157.20s COV: -0.99
[INFO]: t=157.20s RA (from combined map): -71.92+-0.156 mas
[INFO]: t=157.20s DEC (from combined map): -119.48+-0.367 mas
[INFO]: t=157.20s Contrast obtained (mean, min, max): 5.22e-05, 3.94e-05, 6.29e-05
```

The first RA/DEC solution is simply taken from the average of the solutions found on each file.
However, the structure of the chi2 maps is such that are always multiple minimums. When
the chi2 values of successive minimums are similar, the noise between files can make
the solution jump from one minimum to another one between different files, resulting
in a biased astrometry, with large error bars.

The second estimate uses the combined chi2 map (sum of the chi2 maps obtained
with each files) to estimate the best global minimum, which is then used as a starting point
for a gradient descent on each individual chi2 map, to obtain the local minimum closest to
the global minimum. This has the advantage of giving a more robust solution, while also 
allowing to estimate the error bars from the distribution of solutions found on each 
file.

Finally, the range of planet to star contrast values used to fit the planet fringes is displayed,
as a way to quickly see if the algorithm has found a "reasonable planet".

In that example, the difference between the two solutions, and the large error bars on the
first solution are strongly suggestive of a chi2 minimum jump. To check this, the best way
is to request the software to save some images and data. This can be done by tweaking
the configuration file to add a directory where to store this (figdir option):

      figdir: /path/to/a/dir/
      
And rerun the astrometry_reduce.py script

The script will also have added the astrometric solution for each of the planet files in the 
configuration file. If you
reopen the file in a text editor, you should see the in the `planet_ois`
section, each element which was not commented in the `reduce` list now
has an `astrometric_solution`. Note that if you are using exoGravity
with the pyyaml library for managing the YAML configuration files (see
installation section), the lines commented in the YAML file do not
survive read/write cycles. Thus, is using pyyaml, the `reduce` list in
`general` section is now shorter.

Now that each planet files has an astrometric solution, you can move to
the spectrum extaction:

``` bash
spectrum_reduce.py config_file=betaPictorisc2020.yml outputdir=./result
```

When the script is finished, you will have the spectrum stored in 
the exoGravity format in a fits file in "./result".

The fits file contains both the contrast spectrum and the absolute spectrum
of the planet. However, at this point, the absolute spectrum is a copy of the
contrast spectrum, and needs to be multiplied by a stellar spectrum. This can
be done using the normalize_spectrum.py script, which requires a stellar model
and a K-band magnitude. You can use the stellar model file provided in this 
tutorial:

``` bash
normalize_spectrum.py file=./result/spectrum.fits file_model=~/lib/gravity/exoGravity/data/lte080-4.0-0.5.BT-NextGen.7 mag_k=3.495
```

One the spectrum is normalized, you can plot it with the plot_spectrum.py script:

``` bash
plot_spectrum.py file=./result/spectrum.fits 
```

Configuration File
------------------

The reduction done by the exoGravity package is parameterized using a
YAML configuration file. This configuration file contains 3 or 4 main
levels, depending on the type of observation, defined below.

### General

The `general` section contains parameters which are not file-specific,
and which are used throughout the entire reduction.

TO DO